# Handwritten Digit Recognition
This notebook building a classic computer vision application of identifying handwritten digits. 
It's train a simple Convolutional Neural Network on the MNIST dataset.
## Data : MNIST 

###[Banafsheh Hassani](https://www.linkedin.com/in/banafsheh-hassani-7b063a129/)

###[More Projects](https://github.com/BanafshehHassani)

## PyTorch on a single node

This notebook using PyTorch on the Spark driver node for fit the neural network on the MNIST handwritten digit recognition data.

[copied from the PyTorch project](https://github.com/pytorch/examples/blob/53f25e0d0e2710878449900e1e61d31d34b63a9d/mnist/main.py) under the [license](https://github.com/pytorch/pytorch/blob/a90c259edad1ea4fa1b8773e3cb37240df680d62/LICENSE) with slight modifications in comments. Thanks to the developers of PyTorch for this example.

* [Source 1](https://docs.databricks.com/_static/notebooks/deep-learning/pytorch-single-node.html) 
* [Source 2](https://github.com/pytorch/examples/blob/53f25e0d0e2710878449900e1e61d31d34b63a9d/mnist/main.py)
* [license](https://github.com/pytorch/pytorch/blob/a90c259edad1ea4fa1b8773e3cb37240df680d62/LICENSE)

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.dropdown('USE_GPU', 'no', ['no', 'yes'])
USE_GPU = dbutils.widgets.get('USE_GPU') == 'yes'

#Import libraries

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from collections import namedtuple

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
MNIST_DIR = '/tmp/data/mnist'
use_cuda = USE_GPU and torch.cuda.is_available()

Params = namedtuple('Params', ['batch_size', 'test_batch_size', 'epochs', 'lr', 'momentum', 'seed', 'cuda', 'log_interval'])
args = Params(batch_size=64, test_batch_size=1000, epochs=10, lr=0.01, momentum=0.5, seed=1, cuda=use_cuda, log_interval=200)

### Data processing-MNIST  
* Download 
* Shuffle rows 
* Create batches 
* Standardize the features

In [0]:
torch.manual_seed(args.seed)

data_transform_fn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])

train_loader = torch.utils.data.DataLoader(
                 datasets.MNIST(MNIST_DIR, train=True, download=True,
                   transform=data_transform_fn),
               batch_size=args.batch_size, shuffle=True, num_workers=1)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(MNIST_DIR, train=False, 
                       transform=data_transform_fn),
        batch_size=args.test_batch_size, shuffle=True, num_workers=1)


### Build a CNN model

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
      
model = Net()
model.share_memory() 

Out[5]: Net(
 (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
 (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
 (conv2_drop): Dropout2d(p=0.5, inplace=False)
 (fc1): Linear(in_features=320, out_features=50, bias=True)
 (fc2): Linear(in_features=50, out_features=10, bias=True)
)

### Training the model

In [0]:
def train_epoch(epoch, args, model, data_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()      
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.data.item()))


def test_epoch(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()      
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data.item() # sum up batch loss
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(data_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))



Runing training loop over epochs, it evaluate after each.

In [0]:
if args.cuda:
    model = model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
for epoch in range(1, args.epochs + 1):
    train_epoch(epoch, args, model, train_loader, optimizer)
    test_epoch(model, test_loader)

<command-1194993396842650>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
 return F.log_softmax(x)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332552
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.125173
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.772878
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.518417
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.485763
<command-1194993396842652>:25: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
 data, target = Variable(data, volatile=True), Variable(target)
/databricks/python/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
 warnings.warn(warning.format(ret))

Test set: Average loss: 0.1921, Accuracy: 9420/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.364460
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.502845
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.238232
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.279074
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.446922

Test set: Average loss: 0.1240, Accuracy: 9614/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.231387
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.559281
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.292914
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.401911
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.129268

Test set: Average loss: 0.0984, Accuracy: 9698/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.365490
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.236167
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.215550
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.252271
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.479188

Test set: Average loss: 0.0828, Accuracy: 9742/10000 (97%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.199672
Train Epoch: 5 [12800/60000 (21%)]	Loss: 0.149998
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.271687
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.189840
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.446669

Test set: Average loss: 0.0711, Accuracy: 9780/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.149341
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.335459
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.114738
Train Epoch: 6 [38400/60000 (64%)]	Loss: 0.239979
Train Epoch: 6 [51200/60000 (85%)]	Loss: 0.201276

Test set: Average loss: 0.0704, Accuracy: 9786/10000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.175631
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.083417
Train Epoch: 7 [25600/60000 (43%)]	Loss: 0.198340
Train Epoch: 7 [38400/60000 (64%)]	Loss: 0.332557
Train Epoch: 7 [51200/60000 (85%)]	Loss: 0.203750

Test set: Average loss: 0.0610, Accuracy: 9808/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.072501
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.057836
Train Epoch: 8 [25600/60000 (43%)]	Loss: 0.168418
Train Epoch: 8 [38400/60000 (64%)]	Loss: 0.127133
Train Epoch: 8 [51200/60000 (85%)]	Loss: 0.403241

Test set: Average loss: 0.0553, Accuracy: 9835/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.123991
Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.179392
Train Epoch: 9 [25600/60000 (43%)]	Loss: 0.112713
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.129101
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.105125

Test set: Average loss: 0.0536, Accuracy: 9830/10000 (98%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.097767
Train Epoch: 10 [12800/60000 (21%)]	Loss: 0.142445
Train Epoch: 10 [25600/60000 (43%)]	Loss: 0.218235
Train Epoch: 10 [38400/60000 (64%)]	Loss: 0.102758
Train Epoch: 10 [51200/60000 (85%)]	Loss: 0.297475

Test set: Average loss: 0.0516, Accuracy: 9846/10000 (98%)

The key information of training in output:
- In prescribed logging intervals, training process prints out how many samples in the training set, it has gone through in the current epoch, as well as loss of the current training mini-batch. 
- that's obvious over epochs, on testing set, loss decreases plus accuracy increases, suggesting that model is in overal improving, even though at batch level there might be fluctuations.